In [3]:
import os
import pickle
from tqdm import tqdm

root = '/data0/aix23907/gait/CCPG_DenoisingGait'
mismatched = []

print("데이터 정합성 검사 중...")
for id_dir in tqdm(sorted(os.listdir(root))):
    id_path = os.path.join(root, id_dir)
    if not os.path.isdir(id_path): continue
    for ty_dir in os.listdir(id_path):
        ty_path = os.path.join(id_path, ty_dir)
        for vi_dir in os.listdir(ty_path):
            vi_path = os.path.join(ty_path, vi_dir)
            
            fea_p = os.path.join(vi_path, '00-DenoisingFea_96x48.pkl')
            sil_p = os.path.join(vi_path, '01-Sils_96x48.pkl')
            
            if not os.path.exists(fea_p) or not os.path.exists(sil_p):
                print(f"[파일 누락] {vi_path}")
                mismatched.append(vi_path)
                continue
                
            with open(fea_p, 'rb') as f: fea = pickle.load(f)
            with open(sil_p, 'rb') as f: sil = pickle.load(f)
            
            if len(fea) != len(sil):
                print(f"[개수 불일치] {vi_path} | Diffusion: {len(fea)}, Sil: {len(sil)}")
                mismatched.append(vi_path)

print(f"\n총 문제 시퀀스: {len(mismatched)}")

데이터 정합성 검사 중...


  9%|▉         | 18/200 [00:16<02:56,  1.03it/s]

[파일 누락] /data0/aix23907/gait/CCPG_DenoisingGait/018/U1_D0/03_0


 20%|██        | 40/200 [00:38<02:36,  1.02it/s]

[파일 누락] /data0/aix23907/gait/CCPG_DenoisingGait/040/U1_D0/05_0


...

[파일 누락] /data0/aix23907/gait/CCPG_DenoisingGait/178/U0_D0_BG/03_0


 90%|█████████ | 180/200 [02:56<00:17,  1.13it/s]

[파일 누락] /data0/aix23907/gait/CCPG_DenoisingGait/180/U0_D3/05_0


100%|██████████| 200/200 [03:15<00:00,  1.03it/s]


총 문제 시퀀스: 13


In [4]:
import os
import shutil
import pickle
from tqdm import tqdm

# 데이터가 저장된 루트 경로
root = '/data0/aix23907/gait/CCPG_DenoisingGait'
deleted_count = 0

print("정합성 검사 및 불량 폴더 삭제 시작...")

# 1. 모든 ID 폴더 순회
for id_dir in tqdm(sorted(os.listdir(root))):
    id_path = os.path.join(root, id_dir)
    if not os.path.isdir(id_path): continue
    
    for ty_dir in os.listdir(id_path):
        ty_path = os.path.join(id_path, ty_dir)
        for vi_dir in os.listdir(ty_path):
            vi_path = os.path.join(ty_path, vi_dir)
            
            fea_p = os.path.join(vi_path, '00-DenoisingFea_96x48.pkl')
            sil_p = os.path.join(vi_path, '01-Sils_96x48.pkl')
            
            should_delete = False
            reason = ""

            # 체크 1: 파일 존재 여부 확인
            if not os.path.exists(fea_p) or not os.path.exists(sil_p):
                should_delete = True
                reason = "파일 누락"
            else:
                # 체크 2: 프레임 개수 일치 여부 확인
                try:
                    with open(fea_p, 'rb') as f: fea = pickle.load(f)
                    with open(sil_p, 'rb') as f: sil = pickle.load(f)
                    
                    if len(fea) != len(sil):
                        should_delete = True
                        reason = f"개수 불일치 (Fea:{len(fea)}, Sil:{len(sil)})"
                except Exception as e:
                    should_delete = True
                    reason = f"파일 손상 ({e})"

            # 삭제 실행
            if should_delete:
                print(f"❌ 삭제 중: {vi_path} | 사유: {reason}")
                shutil.rmtree(vi_path)
                deleted_count += 1

print(f"\n✅ 완료! 총 {deleted_count}개의 불량 시퀀스 폴더를 삭제했습니다.")

정합성 검사 및 불량 폴더 삭제 시작...


  9%|▉         | 18/200 [00:17<03:04,  1.01s/it]

❌ 삭제 중: /data0/aix23907/gait/CCPG_DenoisingGait/018/U1_D0/03_0 | 사유: 파일 누락


 20%|██        | 40/200 [00:38<02:50,  1.07s/it]

❌ 삭제 중: /data0/aix23907/gait/CCPG_DenoisingGait/040/U1_D0/05_0 | 사유: 파일 누락


...

❌ 삭제 중: /data0/aix23907/gait/CCPG_DenoisingGait/178/U0_D0_BG/03_0 | 사유: 파일 누락


 90%|█████████ | 180/200 [02:52<00:24,  1.22s/it]

❌ 삭제 중: /data0/aix23907/gait/CCPG_DenoisingGait/180/U0_D3/05_0 | 사유: 파일 누락


100%|██████████| 200/200 [03:12<00:00,  1.04it/s]


✅ 완료! 총 13개의 불량 시퀀스 폴더를 삭제했습니다.
